In [3]:
# =====================================================
# 00) MONTAJE DE GOOGLE DRIVE
# =====================================================
from google.colab import drive
drive.mount('/content/drive')

BASE_PATH = '/content/drive/MyDrive/AML_Final_Project/'
print("✅ Google Drive montado")
print(f"📂 Ruta base del proyecto: {BASE_PATH}")

# =====================================================
# 01) LIBRERÍAS
# =====================================================
import pandas as pd
import numpy as np

# =====================================================
# 02) PARÁMETROS
# =====================================================
INPUT_FILE = BASE_PATH + "datasetPrevio.xlsx"
OUTPUT_FILE = BASE_PATH + "dataset_model.parquet"

# Si quieres alinear horas a Perú (recomendado si tus picos son hora local)
USE_LIMA_TZ = True
LIMA_TZ = "America/Lima"

# Completar grilla por local: frecuencia horaria
FREQ = "h"

# Filtrado por horas operativas detectadas por local (reduce ceros estructurales)
FILTER_OPERATING_HOURS = True

# Lags / rolling (arranca con tamaños razonables)
LAGS = [1, 2, 3, 24, 168]         # 1h,2h,3h,1d
ROLL_WINDOWS = [6, 12, 24, 168]   # 6h,12h,24h

# Min porcentaje de pedidos > 0 para considerar una hora como "operativa" (por local)
OPER_HOUR_MIN_POS_RATE = 0.02  # 2% de las observaciones con demanda>0

# =====================================================
# 03) CARGA
# =====================================================
df = pd.read_excel(INPUT_FILE)
print("Shape original:", df.shape)

# =====================================================
# 04) TIPOS Y LIMPIEZA
# =====================================================
df["fecha_creacion_fv"] = pd.to_datetime(df["fecha_creacion_fv"], errors="coerce", utc=True)

for col in ["KM_REAL", "TIEMPO_ENTREGA_RETORNO"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df = df.dropna(subset=["fecha_creacion_fv", "local"])

# Convertir a Lima si aplica (para que la hora sea la real del negocio)
if USE_LIMA_TZ:
    df["fecha_creacion_fv"] = df["fecha_creacion_fv"].dt.tz_convert(LIMA_TZ)

# Bucket horario (ojo: floor('H') -> floor('h'))
df["ts_hour"] = df["fecha_creacion_fv"].dt.floor("h")
df["hora"] = df["ts_hour"].dt.hour

# =====================================================
# 05) AGREGACIÓN BASE (local-hora) SIN REINDEX AÚN
# =====================================================
base = (
    df.groupby(["local", "ts_hour"], as_index=False)
      .agg(
          pedidos=("order_id", "count"),
          km_mean=("KM_REAL", "mean"),
          t_ret_mean=("TIEMPO_ENTREGA_RETORNO", "mean"),
          t_ret_p75=("TIEMPO_ENTREGA_RETORNO",
                     lambda x: np.nanpercentile(x, 75) if np.isfinite(x).any() else np.nan)
      )
)

# =====================================================
# 06) FUNCIÓN: COMPLETAR GRILLA POR LOCAL EN SU RANGO ACTIVO
# =====================================================
def build_full_grid_per_local(base_df: pd.DataFrame) -> pd.DataFrame:
    out = []
    for loc, b in base_df.groupby("local", sort=False):
        min_ts = b["ts_hour"].min()
        max_ts = b["ts_hour"].max()
        hours = pd.date_range(min_ts, max_ts, freq=FREQ, tz=min_ts.tz)  # conserva tz local

        idx = pd.MultiIndex.from_product([[loc], hours], names=["local", "ts_hour"])
        b_full = (
            b.set_index(["local", "ts_hour"])
             .reindex(idx)
             .reset_index()
        )

        # pedidos=0 cuando no hubo registros en esa hora (esto sí es correcto)
        b_full["pedidos"] = b_full["pedidos"].fillna(0).astype(int)

        # Para variables operativas: imputación robusta solo con info del local
        for c in ["km_mean", "t_ret_mean", "t_ret_p75"]:
            med = b[c].median()
            b_full[c] = b_full[c].fillna(med)

        out.append(b_full)

    full = pd.concat(out, ignore_index=True)
    return full

base_full = build_full_grid_per_local(base)

# Fallback global por si algún local tuvo todo NaN en operativas (raro)
for c in ["km_mean", "t_ret_mean", "t_ret_p75"]:
    base_full[c] = base_full[c].fillna(base_full[c].median())

# =====================================================
# 07) (OPCIONAL) FILTRAR HORAS OPERATIVAS POR LOCAL
#     - Evita incluir horas donde el local casi nunca opera
# =====================================================
base_full["hora"] = base_full["ts_hour"].dt.hour

if FILTER_OPERATING_HOURS:
    # tasa de horas con demanda>0 por local y hora
    pos_rate = (
        base_full.assign(pos=(base_full["pedidos"] > 0).astype(int))
                 .groupby(["local", "hora"])["pos"].mean()
                 .reset_index()
                 .rename(columns={"pos": "pos_rate"})
    )

    base_full = base_full.merge(pos_rate, on=["local", "hora"], how="left")
    base_full["pos_rate"] = base_full["pos_rate"].fillna(0)

    # conservar solo horas que superen el umbral
    base_full = base_full[base_full["pos_rate"] >= OPER_HOUR_MIN_POS_RATE].copy()
    base_full.drop(columns=["pos_rate"], inplace=True)

# =====================================================
# 08) FEATURES TEMPORALES
# =====================================================
base_full["dow"] = base_full["ts_hour"].dt.dayofweek
base_full["month"] = base_full["ts_hour"].dt.month
base_full["is_weekend"] = (base_full["dow"] >= 5).astype(int)

# Variables cíclicas
base_full["hora_sin"] = np.sin(2 * np.pi * base_full["hora"] / 24)
base_full["hora_cos"] = np.cos(2 * np.pi * base_full["hora"] / 24)
base_full["dow_sin"] = np.sin(2 * np.pi * base_full["dow"] / 7)
base_full["dow_cos"] = np.cos(2 * np.pi * base_full["dow"] / 7)

# Ordenar para lags/rolling
base_full = base_full.sort_values(["local", "ts_hour"]).reset_index(drop=True)

# =====================================================
# 09) LAGS
# =====================================================
for lag in LAGS:
    base_full[f"pedidos_lag_{lag}h"] = base_full.groupby("local")["pedidos"].shift(lag)

# =====================================================
# 10) ROLLING (sobre la serie ordenada por local)
# =====================================================
for w in ROLL_WINDOWS:
    base_full[f"pedidos_roll_mean_{w}h"] = (
        base_full.groupby("local")["pedidos"]
                 .shift(1)
                 .rolling(window=w, min_periods=max(2, w//3))
                 .mean()
                 .reset_index(level=0, drop=True)
    )
    base_full[f"pedidos_roll_std_{w}h"] = (
        base_full.groupby("local")["pedidos"]
                 .shift(1)
                 .rolling(window=w, min_periods=max(2, w//3))
                 .std()
                 .reset_index(level=0, drop=True)
    )

# =====================================================
# 11) VARIABLE DERIVADA (opcional)
# =====================================================
base_full["flota_requerida_estimada"] = (base_full["pedidos"] * base_full["t_ret_mean"]) / 60

# =====================================================
# 12) FILTRADO FINAL (no mates todo)
#     - Exige solo features clave (por ejemplo lag 1h + 24h + rolling 24h)
# =====================================================
required = [f"pedidos_lag_{min(LAGS)}h", "pedidos_lag_24h", "pedidos_roll_mean_24h"]
df_model = base_full.dropna(subset=required).copy()

print("✅ Shape model-ready:", df_model.shape)
print("Rango fechas:", df_model["ts_hour"].min(), "->", df_model["ts_hour"].max())
print("Locales:", df_model["local"].nunique())

print("\nDistribución pedidos (model-ready):")
print(df_model["pedidos"].describe())

# =====================================================
# 13) GUARDAR
# =====================================================
df_model.to_parquet(OUTPUT_FILE, index=False)
print("\n✅ Dataset model-ready guardado en:")
print(OUTPUT_FILE)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive montado
📂 Ruta base del proyecto: /content/drive/MyDrive/AML_Final_Project/
Shape original: (9504, 11)
✅ Shape model-ready: (19393, 28)
Rango fechas: 2025-02-14 20:00:00-05:00 -> 2025-05-04 18:00:00-05:00
Locales: 25

Distribución pedidos (model-ready):
count    19393.000000
mean         0.443665
std          2.797950
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         81.000000
Name: pedidos, dtype: float64

✅ Dataset model-ready guardado en:
/content/drive/MyDrive/AML_Final_Project/dataset_model.parquet
